In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv("locations_05-19-23.csv")
df.head()

,log_ts,ble_mac,device_mac,ble_rssi
0,2023-05-19 11:00:07,00:1b:66:1b:1c:b7,E0:5A:1B:A0:1E:88,-84
1,2023-05-19 11:00:07,00:4a:88:d0:57:a7,E0:5A:1B:A0:37:D8,-79
2,2023-05-19 11:00:07,00:c3:c4:ed:23:1e,E0:5A:1B:A0:3D:C8,-74
3,2023-05-19 11:00:07,01:4b:e2:74:3e:ce,E0:5A:1B:A0:3D:C8,-85
4,2023-05-19 11:00:16,02:42:49:06:7e:19,CC:50:E3:A8:D9:6C,-77


In [9]:
# Group by timestamps
grouped = df.groupby('log_ts')
ts_df = {ts: df.drop('log_ts', axis=1) for ts, df in grouped}

# At time 2023-05-19 11:00:15
ts_df["2023-05-19 11:00:06"]

,ble_mac,device_mac,ble_rssi
5,02:42:49:06:7e:19,E0:5A:1B:A0:38:C0,-84
10,03:68:d1:11:9c:53,E0:5A:1B:A0:51:9C,-83
12,03:d0:8e:9d:9b:65,E0:5A:1B:A0:40:F8,-73
14,04:08:0a:ba:c7:7c,E0:5A:1B:A0:40:F8,-61
15,04:08:0a:ba:c7:7c,E0:5A:1B:A0:51:9C,-81
...,...,...,...
1031,fc:a7:19:3e:8d:fb,E0:5A:1B:A0:40:F8,-80
1032,fd:ad:38:4f:c2:85,A4:CF:12:43:6A:A0,-76
1035,fe:25:a0:87:f1:34,E0:5A:1B:A0:40:F8,-80
1036,fe:47:62:b9:95:1c,E0:5A:1B:A0:38:C0,-77


In [10]:
# The trick: pivot_table
def transform(df):
    pivot_df = df.pivot_table(index='ble_mac', columns='device_mac', values='ble_rssi', dropna=True)
    pivot_df = pivot_df.reset_index().rename_axis(None, axis=1)
    pivot_df = pivot_df.fillna(-np.inf)     # -inf = no signal
    return pivot_df


# Testing the function on all timestamps with `some user` detected by more than 1 device
badcount = 0
for ts, df in ts_df.items():
    if len(df['device_mac'].unique()) > 1:  # if `some user` is detected by more than 1 device 
        print('timestamp', ts)
        print(transform(df))
        print()
    else:
        badcount += 1
        
print("# of timestamps when the users are detected by less than 2 devices:", badcount)
print('# total timestamps:', len(ts_df))

timestamp 2023-05-19 11:00:06
               ble_mac  3C:71:BF:62:C2:B8  80:7D:3A:BC:C6:30  \
0    02:42:49:06:7e:19               -inf               -inf   
1    03:68:d1:11:9c:53               -inf               -inf   
2    03:d0:8e:9d:9b:65               -inf               -inf   
3    04:08:0a:ba:c7:7c               -inf               -inf   
4    05:b5:54:b1:b2:e4               -inf               -inf   
..                 ...                ...                ...   
230  fc:a7:19:3e:8d:fb               -inf               -inf   
231  fd:ad:38:4f:c2:85               -inf               -inf   
232  fe:25:a0:87:f1:34               -inf               -inf   
233  fe:47:62:b9:95:1c               -inf               -inf   
234  ff:92:ab:70:b2:bf              -78.0               -inf   

     A4:CF:12:43:6A:A0  E0:5A:1B:A0:1A:C0  E0:5A:1B:A0:38:C0  \
0                 -inf               -inf              -84.0   
1                 -inf               -inf               -inf   
2        

Unfortunately, at most times, an external device (user's phone, ipad..) is only detected by 1 or 2 of our sensors.
Makes triangulation hard (inaccuracy++). 

> Hardware team, lower the ble_rssi threshold?

In [11]:
## Maked up test case just to make sure `transform()` works
mydf = pd.DataFrame({
    'ble_mac': [1, 1, 1, 2, 2],
    'device_mac': ['A', 'B', 'C', 'A', 'B'],
    'ble_rssi': [-50, -60, -70, -30, -40]
})

mydf

,ble_mac,device_mac,ble_rssi
0,1,A,-50
1,1,B,-60
2,1,C,-70
3,2,A,-30
4,2,B,-40


In [12]:
transform(mydf)

,ble_mac,A,B,C
0,1,-50.0,-60.0,-70.0
1,2,-30.0,-40.0,-inf
